# List of My Movies

In [8]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile
from timeUtils import clock, elapsed


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-06-24 23:40:27.939143


# Combine Movies

In [5]:
%load_ext autoreload
%autoreload

from combine import combine

from wikipedia import wikipedia
from wikifilm import wikifilm
from oscar import oscars
from razzies import razzies
from rollingstone import rollingstone
from BAFTA import BAFTA
from SAG import SAG
from filmsite import filmsite
from films101 import films101
from AACTA import AACTA
from flops import flops
from canada import Canada
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from ultimatemovierankings import ultimatemovierankings

comb = combine()
comb.setOscarData(oscars(wikipedia()), 10)
comb.setWikiFilmData(wikifilm(), 1)
comb.setRazziesData(razzies(), 1)
comb.setBAFTAData(BAFTA(), 1)
comb.setSAGData(SAG(), 1)
comb.setAACTAData(AACTA(), 1)
comb.setFilmsiteData(filmsite(), 1)
comb.setFilms101Data(films101(), 1)
comb.setFlopsData(flops(), 1)
comb.setCanadaData(Canada(), 1)
comb.setRollingStoneData(rollingstone(), 1)
comb.setRottenTomatoesData(rottentomatoes(), 90)
comb.setBoxOfficeMojoData(boxofficemojo(), 5e6)
comb.setUltimateMovieRankingsData(ultimatemovierankings(), 90)
comb.getData()
comb.saveCorrections(debug=True)
comb.mergeMovies(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 32 Years of Flops Movies
Found 105 Years of Filmsite Movies
Found 98 Years of Wiki Film Movies
Found 94 Years of Rotten Tomatoes Movies
Found 50 Years of AACTA Movies
Found 25 Years of SAG Movies
Found 119 Years of Films101 Movies
Found 39 Years of Box Office Mojo Movies
Found 38 Years of Razzies Movies
Found 67 Years of Ultimate Movie Rankings Movies
Found 84 Years of Oscar Movies
Found 61 Years of Canada Movies
Found 23 Years of Rolling Stone Movies
Found 47 Years of BAFTA Movies
Found Data Between 1900 and 2018
Found 914 old corrections
Found 5 new corrections
Saving data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
Saved data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
  --> This file is 52.6kB.
There are 914 total corrections
Found 17741 movies
	 ('Films101', 8272)
	 ('Box Office Mojo', 3934)
	 ('Oscar', 2267)
	 ('Rotten Tomatoes', 987)
	 ('AACTA',

# Merge My Movies With Combined Movies

In [6]:
from mymovies import mymovies

In [9]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Mon Jun 24, 2019 23:40:31 for Last Run


In [14]:
mine.suggestRenames()

In [15]:
mine.findMyMovies()
_ = clock("Last Run")

Found 6472 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 523.5kB.
Current Time is Mon Jun 24, 2019 23:42:59 for Last Run


In [16]:
### Stopped after 1985. Do 1986 onward
mine.mergeMovies()

In [171]:
show = False
if show:
    renames = mine.getMovieRenames()
    for old,new in renames.items():
        print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)".format(old,new))

In [8]:
mine.searchMyMovies("Ben Is Back", 2019, cutoff=0.6)

Ben Is Back  --->  ['Men In Black 3 [2012]', 'Men In Black [1997]', 'The Way Back [2010]']
moveFile(src="/Volumes/Save/Movies/Men In Black 3 [2012].avi", dst="/Volumes/Save/Movies/Men In Black 3 [2012].avi", debug=True)



In [43]:
#mine.showCombinedMovieStatus(1987)
mine.printMissingMovies(2014)

====================== Missing Movies ======================
Year      Movie                                                                                               
----      -----                                                                                               

2014      12 O'clock Boys                                                                                     
2014      A Pigeon Sat On A Branch Reflecting On Existence                                                    
2014      A Summer's Tale                                                                                     
2014      America                                                                                             
2014      Art And Craft                                                                                       
2014      Becoming Bulletproof                                                                                
2014      Black Souls                             

In [173]:
mine.missingMovies

{1900: {None},
 1901: {None},
 1902: {None},
 1903: {None},
 1904: {None},
 1905: {None},
 1906: {None},
 1907: {None},
 1908: {None},
 1909: {None},
 1910: {None},
 1911: {None},
 1912: {None},
 1913: {None},
 1914: {None},
 1915: {None},
 1916: {None},
 1917: {None},
 1918: {None},
 1919: {None},
 1920: {None},
 1921: {None},
 1922: {None},
 1923: {None},
 1924: {None},
 1925: {None},
 1926: {None},
 1927: {None},
 1928: {None},
 1929: {None},
 1930: {None},
 1931: {None},
 1932: {None},
 1933: {None},
 1934: {None},
 1935: {None},
 1936: {None},
 1937: {None},
 1938: {None},
 1939: {None},
 1940: {None},
 1941: {None},
 1942: {None},
 1943: {None},
 1944: {None},
 1945: {None},
 1946: {None},
 1947: {None},
 1948: {None},
 1949: {None},
 1950: {None},
 1951: {None},
 1952: {None},
 1953: {None},
 1954: {None},
 1955: {None},
 1956: {None},
 1957: {None},
 1958: {None},
 1959: {None},
 1960: {None},
 1961: {None},
 1962: {None},
 1963: {None},
 1964: {None},
 1965: {None},
 1966: {No

In [166]:
printMissingMovies(1995)

====================== Missing Movies ======================
Year      Movie                                                                                               
----      -----                                                                                               

{None}


TypeError: unsupported format string passed to NoneType.__format__

In [51]:

moveFile(src="/Volumes/Save/Movies/Sweeney Todd The Demon Barber Of Fleet Street [2007].mp4", dst="/Volumes/Save/Movies/Sweeney Todd: The Demon Barber Of Fleet Street [2007].mp4", debug=True)

moveFile(src="/Volumes/Download/Movies/The King Of Kong A Fistful Of Quarters [2007].mp4", dst="/Volumes/Download/Movies/The King of Kong: A Fistful of Quarters [2007].mp4", debug=True)

Moving /Volumes/Save/Movies/Sweeney Todd The Demon Barber Of Fleet Street [2007].mp4 to /Volumes/Save/Movies/Sweeney Todd: The Demon Barber Of Fleet Street [2007].mp4
Moving /Volumes/Download/Movies/The King Of Kong A Fistful Of Quarters [2007].mp4 to /Volumes/Download/Movies/The King of Kong: A Fistful of Quarters [2007].mp4


In [40]:

moveFile(src="/Volumes/Seagate/Movies/X The Man With The X Ray Eyes [1963].mp4", dst="/Volumes/Seagate/Movies/X: The Man With The X-Ray Eyes [1963].mp4", debug=True)

Moving /Volumes/Seagate/Movies/X The Man With The X Ray Eyes [1963].mp4 to /Volumes/Seagate/Movies/X: The Man With The X-Ray Eyes [1963].mp4


In [33]:
mine.searchMyMovies("Goldeneye", 1995, cutoff=0.6, num=10)

Goldeneye  --->  ['James Bond Goldeneye [1995]', 'Powder [1995]', 'Volunteers [1985]', 'Jade [1995]', '12 Monkeys [1995]', 'Sleepers [1996]', 'Clueless [1995]', 'Wilde [1997]', 'Se7en [1995]', 'Nixon [1995]']
moveFile(src="/Volumes/Seagate/Movies/James Bond Goldeneye [1995].mkv", dst="/Volumes/Seagate/Movies/James Bond Goldeneye [1995].mkv", debug=True)

